## Kaggle – DataTops®
Luismi ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a Luismi a mejorar ese modelo?

## Métrica: 
Error de raíz cuadrada media (RMSE) es la desviación estándar de los valores residuales (errores de predicción). Los valores residuales son una medida de la distancia de los puntos de datos de la línea de regresión; RMSE es una medida de cuál es el nivel de dispersión de estos valores residuales. En otras palabras, le indica el nivel de concentración de los datos en la línea de mejor ajuste.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## Librerías

In [1]:
import numpy as np
import pandas as pd
import toolbox_ML as tb


from PIL import Image
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import urllib.request

## Datos

In [2]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("./data/train.csv", index_col = 0)
df.index.name = None

In [3]:
df_bck_up = df.copy()

## Exploración de los datos

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    object 
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    object 
 11  Price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB


In [5]:
df.isna().sum()

Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price_in_euros      0
dtype: int64

In [6]:
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [7]:
df.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [8]:
tb.describe_df(df)

,DATA_TYPE,MISSINGS (%),UNIQUE_VALUES,CARDIN (%)
Company,object,0,19,2.08
Product,object,0,480,52.63
TypeName,object,0,6,0.66
Inches,float64,0,17,1.86
ScreenResolution,object,0,36,3.95
Cpu,object,0,107,11.73
Ram,object,0,9,0.99
Memory,object,0,37,4.06
Gpu,object,0,93,10.2
OpSys,object,0,9,0.99


In [9]:
tb.tipifica_variables(df)

,variable,tipo
0,Company,Numérica Discreta
1,Product,Numérica Continua
2,TypeName,Categórica
3,Inches,Numérica Discreta
4,ScreenResolution,Numérica Discreta
5,Cpu,Numérica Discreta
6,Ram,Categórica
7,Memory,Numérica Discreta
8,Gpu,Numérica Discreta
9,OpSys,Categórica


## Procesado de datos

In [10]:
df[df.duplicated()].sort_values("Product")

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
1305,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00
1295,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg,289.00
1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1304,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1283,Lenovo,IdeaPad Y700-15ISK,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,Nvidia GeForce GTX 960M,Windows 10,2.6kg,899.00
1311,Lenovo,IdeaPad Y700-15ISK,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,Nvidia GeForce GTX 960M,Windows 10,2.6kg,899.00
1310,Dell,Inspiron 3552,Notebook,15.6,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,500GB HDD,Intel HD Graphics,Windows 10,2.20kg,379.00
1285,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R5 M430,Linux,2.3kg,805.99
1294,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i3 7100U 2.4GHz,6GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.3kg,459.00
1313,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R5 M430,Linux,2.3kg,805.99


In [11]:
# Viendo esos duplicados uno a uno, procedemos a eliminar los que realmente están duplicados utilizando su índice:
df.drop(index = [1313, 1318, 1320, 1315, 1316, 1303], inplace = True)

In [12]:
df["inches"] = df.Inches.astype(float)
df['ram_gb'] = df['Ram'].str.replace('GB', '').astype(int)
df["weight_kg"] = df['Weight'].str.replace('kg', '').astype(float)
df.drop(columns = ["Inches", "Ram", "Weight"], inplace = True)

In [ ]:
# Ordenamos las features categóricas de menor precio de median a mayor precio para poder convertirlas en numéricas:
# categories_company = df.groupby(df["Company"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_typename = df.groupby(df["TypeName"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_resolution = df.groupby(df["ScreenResolution"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_cpu = df.groupby(df["Cpu"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_memory = df.groupby(df["Memory"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_gpu = df.groupby(df["Gpu"])["Price_in_euros"].median().sort_values().index.to_list()

# total_cat = [categories_company, categories_resolution, categories_cpu, categories_memory, categories_gpu]
# cols = ["Company", "ScreenResolution", "Cpu", "Memory", "Gpu"]

# enc = OrdinalEncoder(categories = total_cat, dtype= int, handle_unknown = "use_encoded_value", unknown_value = -1)
# df[cols] = enc.fit_transform(df[cols])
# df.head()

,Company,Product,TypeName,ScreenResolution,Cpu,Memory,Gpu,OpSys,Price_in_euros,inches,ram_gb,weight_kg
755,8,250 G6,Notebook,11,33,20,43,Windows 10,539.00,15.6,8,1.86
618,7,Inspiron 7559,Gaming,11,77,11,45,Windows 10,879.01,15.6,16,2.59
909,8,ProBook 450,Notebook,11,67,11,40,Windows 10,900.00,15.6,8,2.04
2,11,Macbook Air,Ultrabook,10,58,14,38,macOS,898.94,13.3,8,1.34
286,7,Inspiron 3567,Notebook,11,32,11,16,Linux,428.00,15.6,4,2.25


In [22]:
feat_sel = tb.get_features_cat_regression(df, "Price_in_euros")
print(feat_sel)

['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Memory', 'Gpu', 'OpSys']


In [20]:
tb.get_features_num_regression(df, "Price_in_euros", 0.6)

[]

In [23]:
df = pd.get_dummies(df, columns = feat_sel, dtype = int)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 906 entries, 755 to 229
Columns: 791 entries, Price_in_euros to OpSys_macOS
dtypes: float64(3), int32(788)
memory usage: 2.8 MB


Nuestro target es la columna `Price_in_euros`

In [25]:
target = "Price_in_euros"

-----------------------------------------------------------------------------------------------------------------

## Modelado

### 1. Definir X e y

In [26]:
X = df.drop(columns = [target])
y = df[target]

### 2. Dividir X_train, X_test, y_train, y_test

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [28]:
X_train.head()

,inches,ram_gb,weight_kg,Company_Acer,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,...,Gpu_Nvidia Quadro M620M,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS
153,15.6,16,2.56,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
275,13.3,8,1.37,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
111,15.6,16,2.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
207,15.6,16,3.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
771,12.3,8,1.10,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [29]:
y_train.head()

153    1249.26
275    1958.90
111    1299.00
207    1749.00
771    1559.00
Name: Price_in_euros, dtype: float64

### 3. Baseline de modelos


In [30]:
rfmodel = RandomForestRegressor(max_depth = 10)

rfmodel.fit(X_train, y_train)

RandomForestRegressor(max_depth=10)

In [31]:
y_pred = rfmodel.predict(X_test)

### 4. Sacar métricas, valorar los modelos 

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

In [32]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

304.1190063676359


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 1. Carga los datos de `test.csv` para predecir.


In [39]:
X_pred = pd.read_csv("data/test.csv", index_col = 0)
X_pred.index.name = None
X_pred.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
209,Lenovo,Legion Y520-15IKBN,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,No OS,2.4kg
1281,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg
1168,Lenovo,V110-15ISK (i3-6006U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,No OS,1.9kg
1231,Dell,Inspiron 7579,2 in 1 Convertible,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.191kg
1020,HP,ProBook 640,Notebook,14.0,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.95kg


In [34]:
X_pred.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
820,MSI,GE72MVR 7RG,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.9kg
948,Toshiba,Tecra Z40-C-12X,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.47kg
483,Dell,Precision M5520,Workstation,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia Quadro M1200,Windows 10,1.78kg
1017,HP,Probook 440,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,Intel HD Graphics 620,Windows 10,1.64kg
421,Asus,ZenBook Flip,2 in 1 Convertible,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.27kg


In [35]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 209 to 421
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   Product           391 non-null    object 
 2   TypeName          391 non-null    object 
 3   Inches            391 non-null    float64
 4   ScreenResolution  391 non-null    object 
 5   Cpu               391 non-null    object 
 6   Ram               391 non-null    object 
 7   Memory            391 non-null    object 
 8   Gpu               391 non-null    object 
 9   OpSys             391 non-null    object 
 10  Weight            391 non-null    object 
dtypes: float64(1), object(10)
memory usage: 36.7+ KB


 ## 2. Replicar el procesado para ``test.csv``

In [40]:
X_pred["inches"] = X_pred.Inches.astype(float)
X_pred['ram_gb'] = X_pred['Ram'].str.replace('GB', '').astype(int)
X_pred["weight_kg"] = X_pred['Weight'].str.replace('kg', '').astype(float)
X_pred.drop(columns = ["Inches", "Ram", "Weight"], inplace = True)

In [41]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 209 to 421
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   Product           391 non-null    object 
 2   TypeName          391 non-null    object 
 3   ScreenResolution  391 non-null    object 
 4   Cpu               391 non-null    object 
 5   Memory            391 non-null    object 
 6   Gpu               391 non-null    object 
 7   OpSys             391 non-null    object 
 8   inches            391 non-null    float64
 9   ram_gb            391 non-null    int32  
 10  weight_kg         391 non-null    float64
dtypes: float64(2), int32(1), object(8)
memory usage: 35.1+ KB


In [42]:
# # Ordenamos las features categóricas de menor precio de median a mayor precio para poder convertirlas en numéricas:
# categories_company = df.groupby(df["Company"])["Price_in_euros"].median().sort_values().index.to_list()
# # categories_typename = df.groupby(df["TypeName"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_resolution = df.groupby(df["ScreenResolution"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_cpu = df.groupby(df["Cpu"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_memory = df.groupby(df["Memory"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_gpu = df.groupby(df["Gpu"])["Price_in_euros"].median().sort_values().index.to_list()

# total_cat = [categories_company, categories_resolution, categories_cpu, categories_memory, categories_gpu]
# cols = ["Company", "ScreenResolution", "Cpu", "Memory", "Gpu"]

# enc = OrdinalEncoder(categories = total_cat, dtype= int)
# X_pred[cols] = enc.transform(X_pred[cols])

# Con la ram, el sistema operativo y el typename, vamos a hacer un get_dummies:
cols_dummies = X_train.columns
X_pred = pd.get_dummies(X_pred, columns = feat_sel, dtype = int).reindex(columns = cols_dummies, fill_value=0)

In [ ]:
# X_pred.head()

,Company,ScreenResolution,Cpu,Memory,Gpu,inches,weight_kg,TypeName_2 in 1 Convertible,TypeName_Gaming,TypeName_Netbook,...,OpSys_macOS,ram_gb_2,ram_gb_4,ram_gb_6,ram_gb_8,ram_gb_12,ram_gb_16,ram_gb_24,ram_gb_32,ram_gb_64
209,5,11,82,30,70,15.6,2.400,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1281,3,3,8,7,2,15.6,2.400,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1168,5,3,32,11,43,15.6,1.900,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1231,7,14,52,20,41,15.6,2.191,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1020,8,11,52,20,41,14.0,1.950,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [43]:
predictions_submit = rfmodel.predict(X_pred)
predictions_submit

array([1742.39043587,  365.08537166,  417.09898228, 1026.19457251,
        807.94443067,  580.30441425,  729.5318613 ,  940.27993971,
       1342.11657625,  365.84509318, 2307.51018063, 1418.99288117,
        566.75913907, 1608.25204968,  707.38014571,  581.1590534 ,
       2304.16747016, 1426.62469573, 1683.48093723,  590.77566163,
       1723.70581678,  370.96841415,  720.3640582 , 1524.80900336,
        552.28911837,  738.79160916,  671.20158929,  566.86010468,
       2774.2512635 , 1018.11940646, 2306.35796758,  458.03979051,
        705.42014844, 3290.9315    , 2050.98937612, 1648.97794701,
        556.50627941, 1643.89357631,  948.72324606, 1487.42139071,
        808.38084386,  869.02698494,  520.81670674, 1101.06463988,
       1199.07554686, 1085.3226404 ,  999.31146108,  561.46387055,
        822.28302345,  512.4348546 , 1850.26336618,  736.10327384,
       1132.88421197,  505.25938009, 1644.35310743, 1801.30292601,
        593.8090976 ,  772.75838858, 1002.9493749 ,  606.13372

### 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [44]:
sample = pd.read_csv("data/sample_submission.csv")

In [45]:
sample.head()

,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6


In [46]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [47]:
#¿Cómo creamos la submission?
submission = pd.DataFrame({"laptop_ID": X_pred.index,
                        "Price_in_euros": predictions_submit})

In [48]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1742.390436
1,1281,365.085372
2,1168,417.098982
3,1231,1026.194573
4,1020,807.944431


In [49]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [50]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [51]:
chequeador(submission)

You're ready to submit!
